In [1]:
import numpy as np
import os
import glob
import torch
from data_loaders.dataloader3d import get_dataloader, load_data, MotionDataset, TestDataset

In [46]:
def subtract_root(data):
    #only after frames have been cut
    root = (data[0,8,:]+data[0, 9, :])/2
    data=np.delete((data - root), (1,8), axis=1)
    return data

In [47]:
def drop_duplicate_frames(data):
    """
    Drop frames where all 25 rows are identical.
    
    Parameters:
    -----------
    data : numpy.ndarray
        Array with shape (frames, 25, 5)
    
    Returns:
    --------
    numpy.ndarray
        Filtered array with duplicate frames removed
    """
    first_row = data[:, 0:1, :]  # Shape: (frames, 1, 5)
    all_rows_same = np.all(data == first_row, axis=(1,2))

    mask = ~all_rows_same
    return data[mask]

In [4]:
#a,b = load_data("mydataset", split='train', keypointtype='openpose')
a,b,c = load_data("test_dataset", split='test', keypointtype='openpose')

In [ ]:
a[0].shape, b[0].shape, c[0].shape

TypeError: 'NoneType' object is not subscriptable

In [8]:
dataset = TestDataset(
    "gait",
    a,
    b,
    betas=c,
    input_motion_length=296,
)

In [5]:
dataset = MotionDataset(
    "gait",
    a,
    b,
    input_motion_length=296,
)

In [9]:
dataloader = get_dataloader(
    dataset, "train", batch_size=1, num_workers=2
)

In [10]:
print(len(dataset))        # number of samples
print(type(dataset))  

6
<class 'data_loaders.dataloader3d.TestDataset'>


In [25]:
img, label, betas = dataset[7]
print(img.shape, label.shape, betas.shape)  # shapes of the first sample

torch.Size([296, 135]) torch.Size([296, 135]) (82, 11)


In [28]:
for batch in dataloader:
    images, labels,betas = batch
    print(images.shape, labels.shape, betas.shape)  # shapes of the batch
    break  # Just process the first batch for demonstration

torch.Size([1, 296, 135]) torch.Size([1, 296, 135]) torch.Size([1, 82, 11])


In [ ]:
i=2
print(a[i].shape)
print(b[i].shape)

torch.Size([91, 135])
torch.Size([75, 135])


In [ ]:
minx=5000
maxx=-1000
miny=5000
maxy=-1000
minz=5000
maxz=-1000
root="mydataset"
shapes=[]

for patient in os.listdir(root):
    for file in sorted(os.listdir(os.path.join(root, patient))):

        mypath=os.path.join(root, patient, file)
        mypath=os.path.join(mypath,"split_subjects", "0", "keypoints_3d", "smpl-keypoints-3d_cut.npy")
        #print(mypath)
        
        try:
            data=np.load(mypath)
        except:
            print("Could not load:", mypath)    
            continue
        data=drop_duplicate_frames(data)
        shapes.append(data.shape[0])
        #print(data.shape)
        data=subtract_root(data)
        datax=data[...,0]
        datay=data[...,1]
        dataz=data[...,2]

        curminx=np.min(datax)
        curmaxx=np.max(datax)
        curminy=np.min(datay)
        curmaxy=np.max(datay)
        curminz=np.min(dataz)
        curmaxz=np.max(dataz)

        if curminx < minx:
            minx = curminx
        if curmaxx > maxx:
            maxx = curmaxx
        if curminy < miny:
            miny = curminy
        if curmaxy > maxy:
            maxy = curmaxy
        if curminz < minz:
            minz = curminz
        if curmaxz > maxz:
            maxz = curmaxz
print("Overall min and max values::")
print(minx, maxx)
print(miny, maxy)
print(minz, maxz)
print("Shapes:", max(shapes), min(shapes))

Could not load: mydataset\gait_682\20250919_c2_a3_Take2\split_subjects\0\keypoints_3d\smpl-keypoints-3d_cut.npy
Could not load: mydataset\gait_682\20250919_c2_a3_Take3\split_subjects\0\keypoints_3d\smpl-keypoints-3d_cut.npy
Could not load: mydataset\gait_682\20250919_c2_a4_Take1\split_subjects\0\keypoints_3d\smpl-keypoints-3d_cut.npy
Could not load: mydataset\gait_682\20250919_c2_a4_Take2\split_subjects\0\keypoints_3d\smpl-keypoints-3d_cut.npy
Could not load: mydataset\gait_682\20250919_c2_a5_Take1\split_subjects\0\keypoints_3d\smpl-keypoints-3d_cut.npy
Could not load: mydataset\gait_682\20250919_c2_a5_Take2\split_subjects\0\keypoints_3d\smpl-keypoints-3d_cut.npy
Overall min and max values::
-1.6090182703484142 0.38673450951507693
-0.9003782868966556 0.8199114286130842
-2.930964477812297 0.5061700658363381
Shapes: 238 31


In [12]:
data = np.load('mydataset/gait_753/20250617_c1_a2_Take1/split_subjects/0/keypoints_3d/smpl-keypoints-3d.npy')
print(data.shape)
data= np.load('mydataset/gait_753/20250617_c1_a1_Take1/split_subjects/0/fit-smplx/smpl-keypoints-3d_cut.npy')

print("Keys inside:", data.files)
for k in data.files:
    print(k, data[k].shape)

(53, 25, 5)


AttributeError: 'numpy.ndarray' object has no attribute 'files'

In [14]:
data= np.load('mydataset/gait_700/20250820_c1_a1_Take1/split_subjects/0/fit-smplx/smplx-params.npz')

print("Keys inside:", data.files)
for k in data.files:
    print(k, data[k].shape)

Keys inside: ['betas', 'global_orient', 'body_pose', 'transl', 'left_hand_pose', 'right_hand_pose', 'jaw_pose', 'leye_pose', 'reye_pose', 'expression']
betas (121, 10)
global_orient (121, 3)
body_pose (121, 63)
transl (121, 3)
left_hand_pose (121, 12)
right_hand_pose (121, 12)
jaw_pose (121, 3)
leye_pose (121, 3)
reye_pose (121, 3)
expression (121, 10)


In [11]:
data= np.load('results/generated_motion_0.npy')
print(data.shape)

(240, 69)


In [ ]:
from utils.transformation_sixd import smplx_to_6d, sixd_to_smplx

In [ ]:
x=smplx_to_6d("mydataset\\740\\20251002_c1_a1_Take1\\split_subjects\\0\\fit-smplx\\smplx-params.npz", "test/")

In [ ]:
sixd_to_smplx("test/motion_6d_with_transl.npz", "test/", "smpl_models/")

c:\Users\Rui\Vorlesungskript\Master\Thesis\Thesis_project\utils\utils_transform.py:54: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Cross.cpp:67.)
  rot_vec_3 = torch.cross(rot_vec_1, rot_vec_2)


Performing forward kinematics to calculate joint positions...
Using generic SMPL-X model for forward kinematics.
Extracting joints for 109 frames...

Successfully reconstructed 3D keypoints.
Output shape: (109, 22, 3)
Saved result to: test/reconstructed_keypoints_3d_ref.npy


In [ ]:
data=np.load("012314.npy")

In [ ]:
data.shape

(170, 22, 3)

In [ ]:
import numpy as np

def generate_keypoints_only_mask(keypoints_path, height_thresh=0.03, vel_thresh=0.05, video_fps=30):
    """
    Determines foot contact purely from kinematics (Keypoints), ignoring the Force Plate.
    
    Criteria for Contact:
    1. Height: Joint Y-coordinate is close to the ground floor.
    2. Velocity: Joint is not moving significantly (Stance phase is static).

    Args:
        keypoints_path (str): Path to .npy keypoints (frames, 25, 3).
        height_thresh (float): Max distance from floor (e.g., 0.03m).
        vel_thresh (float): Max velocity to be considered "static" (e.g., 0.05 m/frame).
        video_fps (int): Video frame rate.

    Returns:
        np.ndarray: Binary mask of shape (frames, 25).
    """
    
    # 1. Load Data
    try:
        keypoints = np.load(keypoints_path) # Shape (F, 25, 3)
        n_frames = keypoints.shape[0]
    except Exception as e:
        print(f"Error: {e}")
        return None

    # 2. Calculate the Floor (Ground Truth)
    # We assume the lowest 5% of ALL foot points in the entire file represent the floor.
    # This is robust to outliers.
    foot_indices = [19, 20, 21, 22, 23, 24] # L/R BigToe, SmallToe, Heel
    all_foot_y = keypoints[:, foot_indices, 1]
    
    # Filter out zeros (undetected points)
    valid_y = all_foot_y[all_foot_y != 0]
    if len(valid_y) == 0:
        print("No valid foot keypoints found.")
        return np.zeros((n_frames, 25), dtype=int)
        
    floor_level = np.percentile(valid_y, 5) # 5th percentile is the estimated floor
    print(f"Auto-detected Floor Level: {floor_level:.4f}")

    velocity = np.linalg.norm(np.diff(keypoints, axis=0, prepend=keypoints[0:1]), axis=2)
    
    is_low = keypoints[:, :, 1] < (floor_level + height_thresh)

    is_static = velocity < vel_thresh
    
    is_detected = keypoints[:, :, 1] != 0

    contact_mask = is_low & is_static & is_detected

    final_mask = np.zeros_like(contact_mask, dtype=int)
    final_mask[:, foot_indices] = contact_mask[:, foot_indices].astype(int)

    return final_mask


In [ ]:
path="mydataset/740/20251002_c1_a3_Take2/split_subjects/0/keypoints_3d/smpl-keypoints-3d.npy"
mask = generate_keypoints_only_mask(path, height_thresh=0.03, vel_thresh=0.04)
data=np.load(path)
masked_data = data * mask[:, :, np.newaxis]
np.save('masked_keypoints.npy', masked_data)
print('saved')


Auto-detected Floor Level: 0.3254
saved


In [ ]:
path="mydataset/gait_740/20251002_c1_a3_Take2/split_subjects/0/keypoints_3d/smpl-keypoints-3d.npy"
data=np.load(path)

In [ ]:
data=np.delete(data, (1,8), axis=1)  # remove the second joint (index 1)

In [ ]:
data.shape

(134, 23, 5)

In [ ]:
import numpy as np

def get_transformation_matrix(v):
    """
    Creates a matrix that rotates v to the X-axis, mirrors Z, 
    and rotates back.
    """
    # 1. Define input and target
    v = np.array(v, dtype=float)
    target = np.array([1, 0, 0], dtype=float)
    
    # Normalize input
    norm_v = np.linalg.norm(v)
    if norm_v == 0: return np.eye(3)
    a = v / norm_v
    b = target

    # 2. Compute Rotation (Rodrigues' Formula)
    # Axis of rotation (k)
    k = np.cross(a, b)
    s = np.linalg.norm(k) # sin of angle
    c = np.dot(a, b)      # cos of angle

    if s == 0:
        # Vector is already on x-axis (parallel)
        if c > 0: return np.diag([1, 1, -1]) # Just mirror
        # If anti-parallel (-x), we usually just flip indices, 
        # but technically needs 180 rotation. Simplified here:
        return np.diag([1, 1, -1]) 

    # Skew-symmetric matrix K
    K = np.array([
        [0, -k[2], k[1]],
        [k[2], 0, -k[0]],
        [-k[1], k[0], 0]
    ])

    # Rotation Matrix (aligns v -> x)
    # Note: We want R that takes v to x. 
    # The formula R = I + K + ... rotates a to b.
    R = np.eye(3) + K + (K @ K) * ((1 - c) / (s**2))

    # 3. Mirror Matrix (Mirror across XY plane means Z -> -Z)
    M = np.diag([1, 1, -1])

    # 4. Combine: Un-rotate * Mirror * Rotate
    # Note: If R takes v->x, then R.T takes x->v.
    # Order depends on if we view R as transforming the basis or the vector.
    # Standard: T_final = R.T @ M @ R
    return R.T @ M @ R

# --- Usage ---
input_vector = [1, 1, 1] # The vector defining the orientation
points_to_transform = [
    [2, 2, 2],
    [0, 5, 0],
    [1, 1, 1]
]

transform_matrix = get_transformation_matrix(input_vector)

In [41]:
data=np.load("mydataset/gait_766\\20251001_c2_a3_Take3\\split_subjects\\0\\keypoints_3d\\smpl-keypoints-3d_cut.npy")

In [42]:
root=data[0,2,:]+data[0, 5, :]/2
dest=data[44,2,:]+data[44, 5, :]/2
vec=dest - root
vec=vec[:3]
transform_matrix = get_transformation_matrix(vec)

In [43]:
transformed_data = data[..., :3] @ transform_matrix.T

In [44]:
data[..., :3] = transformed_data
np.save("mydataset/gait_766\\20251001_c2_a3_Take3\\split_subjects\\0\\keypoints_3d\\smpl-keypoints-3d_cut.npy", data)

In [ ]:
import os
import numpy as np
from scipy.spatial.transform import Rotation as R

def rotate_smplx_params_y_180(smplx_path: str):
    """
    Loads an SMPL-X parameters file, rotates the global orientation and translation
    180 degrees around the Y-axis, and saves the result back to the file.

    Args:
        smplx_path (str): Path to the .npz or .npy file containing the dictionary 
                          of SMPL-X parameters.
    """
    if not os.path.exists(smplx_path):
        print(f"Error: File not found at '{smplx_path}'")
        return

    print(f"Loading SMPL-X params from: {smplx_path}")
    
    # Load the data
    # allow_pickle is needed if the .npy contains a dictionary object
    data = np.load(smplx_path, allow_pickle=True)
    
    # Convert to a mutable dictionary based on file type
    if isinstance(data, np.lib.npyio.NpzFile):
        # If it's a .npz (zipped), convert to dict
        params = dict(data)
    elif data.ndim == 0 and data.dtype == 'O':
        # If it's a 0-d array wrapping a dict (common in some datasets)
        params = data.item()
    else:
        # If it's already a dict or structured array
        params = dict(data)

    # ---------------------------------------------------------
    # 1. Rotate Translation (transl)
    # ---------------------------------------------------------
    if 'transl' in params:
        # Transformation: (x, y, z) -> (-x, y, -z)
        # We multiply x and z by -1
        transl = params['transl'].copy()
        transl[:, 0] *= -1  # Invert X
        transl[:, 2] *= -1  # Invert Z
        params['transl'] = transl

    # ---------------------------------------------------------
    # 2. Rotate Global Orientation (global_orient)
    # ---------------------------------------------------------
    if 'global_orient' in params:
        global_orient = params['global_orient'] # Shape (N, 3) - Axis Angle
        
        # Convert current axis-angle to Rotation object
        rot_original = R.from_rotvec(global_orient)
        
        # Create a 180-degree rotation around Y-axis
        rot_180_y = R.from_euler('y', 180, degrees=True)
        
        # Apply the rotation: New = Rot180 * Old
        # (We left-multiply to apply the rotation to the global frame)
        rot_new = rot_180_y * rot_original
        
        # Convert back to axis-angle and ensure correct datatype
        params['global_orient'] = rot_new.as_rotvec().astype(global_orient.dtype)

    # ---------------------------------------------------------
    # Save Output
    # ---------------------------------------------------------
    base_path, ext = os.path.splitext(smplx_path)
    
    if ext == '.npz':
        np.savez(smplx_path, **params)
    else:
        np.save(smplx_path, params)
        
    print(f" -> Modified and saved to: {smplx_path}")

In [5]:
rotate_smplx_params_y_180("mydataset/gait_753/20250617_c1_a4_Take2/split_subjects/0/fit-smplx/smplx-params.npz")

Loading SMPL-X params from: mydataset/gait_753/20250617_c1_a4_Take2/split_subjects/0/fit-smplx/smplx-params.npz
 -> Modified and saved to: mydataset/gait_753/20250617_c1_a4_Take2/split_subjects/0/fit-smplx/smplx-params_rotated.npz
